#Code de Recurrent Neural network #

This is the base code to create a model to predict stock prices using the LSTM 

In [2]:
!pip install yfinance

!pip install mplfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import yfinance as yf


##Getting the Data##

In [13]:
from datetime import date
startDate='2022-06-22'
endDate='2022-06-24'
differenceDay=6

startDate=date.fromisoformat(startDate)
endDate=date.fromisoformat(endDate)





In [14]:


for i in range(0,40):
  startDate=np.datetime64(startDate) + np.timedelta64(differenceDay, 'D')
  startDate=str(startDate)

  endDate=np.datetime64(endDate) + np.timedelta64(differenceDay, 'D')
  endDate=str(endDate)

  df = yf.download('TWTR',start = startDate, end = endDate)
  df_f=df
  df_f.append(df_f)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- TWTR: Data doesn't exist for startDate = 1657929600, endDate = 1658102400
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- TWTR: Data doesn't exist for startDate = 1658448000, endDate = 1658620800
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- TWTR: Data doesn't exist for startDate = 1658966400, endDate = 1659139200
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- TWTR: Data doesn't exist for startDate = 1659484800, endDate = 1659657600
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- TWTR: Data doesn't exist for startDate = 1

In [ ]:
#df = yf.download('TWTR',start = '2022-06-17', end = '2022-06-24', interval='1m')

In [ ]:
df=df.drop('Adj Close', axis=1)

#Separate dates for future plotting
Data_dates = df.index
Data_dates=pd.to_datetime(Data_dates)
Data_dates=Data_dates.tz_localize(None)


In [ ]:
#Getting the columns name
cols = list(df)[0:5]
print(cols)

In [ ]:
#New dataframe with only training data - 5 columns
df_for_training = df[cols].astype(float)
print(type(df_for_training))
print(df_for_training.shape)

## Data Analyst ##

## Scaling and splitting data ##

In [ ]:
print(df.shape)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaler = scaler.fit(df_for_training)
DS_raw_scaled = scaler.transform(df_for_training)
print(DS_raw_scaled.shape)

Data set was Scaled and then splitted in columns

In [ ]:
#Scaling for OutPuts "Y" colums
DS_raw_Open_scaled=DS_raw_scaled[:,[0]] #selecting colums from a numpy array
DS_raw_High_scaled=DS_raw_scaled[:,[1]]
DS_raw_Low_scaled=DS_raw_scaled[:,[2]]
DS_raw_Close_scaled=DS_raw_scaled[:,[3]]
DS_raw_Volume_scaled=DS_raw_scaled[:,[4]]

print(DS_raw_Open_scaled.shape)

In [ ]:
#Empty lists to be populated using formatted training data
DS_finished_X = []
DS_finished_Open_Y = []
DS_finished_High_Y = []
DS_finished_Low_Y = []
DS_finished_Close_Y = []
DS_finished_Volume_Y = []

n_future = 1   # Number of units(day, min, hour, etc..) we want to look into the future based on the past days.
n_past =30

In [ ]:
# Creatig the data batches, each one with 30d days

for i in range(n_past, len(DS_raw_scaled) - n_future +1):
  DS_finished_X.append(DS_raw_scaled[i - n_past:i, 0:DS_raw_scaled.shape[1]])
  DS_finished_Open_Y.append(DS_raw_Open_scaled[i:i+1, 0:DS_raw_scaled.shape[1]])
  DS_finished_High_Y.append(DS_raw_High_scaled[i:i+1, 0:DS_raw_scaled.shape[1]])
  DS_finished_Low_Y.append(DS_raw_Low_scaled[i:i+1, 0:DS_raw_scaled.shape[1]])
  DS_finished_Close_Y.append(DS_raw_Close_scaled[i:i+1, 0:DS_raw_scaled.shape[1]])
  DS_finished_Volume_Y.append(DS_raw_Volume_scaled[i:i+1, 0:DS_raw_scaled.shape[1]])

In [ ]:
DS_finished_X, DS_finished_Open_Y, DS_finished_High_Y, DS_finished_Low_Y =  np.array(DS_finished_X), np.array(DS_finished_Open_Y), np.array(DS_finished_High_Y), np.array(DS_finished_Low_Y)
DS_finished_Close_Y, DS_finished_Volume_Y = np.array(DS_finished_Close_Y), np.array(DS_finished_Volume_Y)

In [ ]:
print('trainX shape == {}.'.format(DS_finished_X.shape))
print('trainY_Open shape == {}.'.format(DS_finished_Open_Y.shape))
print('trainY_High shape == {}.'.format(DS_finished_High_Y.shape))
print('trainY_Low shape == {}.'.format(DS_finished_Low_Y.shape))
print('trainY_Close shape == {}.'.format(DS_finished_Close_Y.shape))
print('trainY_Volume shape == {}.'.format(DS_finished_Volume_Y.shape))




In [ ]:
DS_dates_finished_X = []

In [ ]:
### Creating the data dates batches
for a in range(n_past, len(Data_dates) - n_future +1):
  DS_dates_finished_X.append(Data_dates[a - n_past:a])
  #DS_dates_finished_X.append(Data_dates[a - n_past:a, 0:Data_dates.shape[1]])
DS_dates_finished_X = np.array(DS_dates_finished_X)
DS_dates_finished_X.shape

In [ ]:
#This Function split dataset with shape (z,x, y) and is splited in z
def Split3DimData(DataSet,percentageTrainig):
  percentageTrainDataset = 0

  percentageTrainDataset=int((DataSet.shape[0]*percentageTrainig)/100)
  DataSetSplittedTraining=DataSet[0:percentageTrainDataset]
  DataSetSplittedTesting= DataSet[percentageTrainDataset:]

  return DataSetSplittedTraining, DataSetSplittedTesting

### Let's split data ###
It's going to be used 70% of data to train and 30% to test the model

In [ ]:

percentageTrainingData= 70
#####   training data, testing data  ####
trainX, testingX = Split3DimData(DS_finished_X,percentageTrainingData)
trainY_Open, testingY_Open = Split3DimData(DS_finished_Open_Y,percentageTrainingData)
trainY_High, testingY_High = Split3DimData(DS_finished_High_Y,percentageTrainingData)
trainY_Low, testingY_Low = Split3DimData(DS_finished_Low_Y,percentageTrainingData)
trainY_Close, testingY_Close = Split3DimData(DS_finished_Close_Y,percentageTrainingData)
trainY_Volume,testingY_Volume = Split3DimData(DS_finished_Volume_Y,percentageTrainingData)

##Validated data set, im getting the spected result

print(testingX.shape)
print(trainX.shape)

### Let's Split data dates ###

In [ ]:
train_Dates, testing_Dates = Split3DimData(Data_dates,percentageTrainingData)

print(train_Dates.shape)
print(testing_Dates.shape)

## Model creation ##

In [ ]:
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.layers.core import Activation

In [ ]:
keras.backend.clear_session()  # Reseteo sencillo

#---------Layes are created


inputs=keras.Input(shape=(30,5))


LSTM_Layer1=keras.layers.LSTM(90, input_shape=(30,5), return_sequences=True, activation='sigmoid')(inputs)

#x=Dropout_layer1=keras.layers.Dropout(0.2)(x)
LSTM_Layer2=keras.layers.LSTM(90, return_sequences=False)(LSTM_Layer1)

Dropout_layer3=keras.layers.Dropout(0.2)(LSTM_Layer2)# modify

In [ ]:
#---------------------------Outputs
dense2=keras.layers.Dense(1)(Dropout_layer3)
dense2_2=keras.layers.Dense(1)(Dropout_layer3)
dense2_3=keras.layers.Dense(1)(Dropout_layer3)
dense2_4=keras.layers.Dense(1)(Dropout_layer3)
dense2_5=keras.layers.Dense(1)(Dropout_layer3)

In [ ]:
#-------Layers outputs are linked

outputs=dense2
outputs2=dense2_2
outputs3=dense2_3
outputs4=dense2_4
outputs5=dense2_5

In [ ]:
#-----The model it's created

model=keras.Model(inputs=inputs, outputs=[outputs,outputs2,outputs3,outputs4,outputs5], name='Prices_Forcasting')
#model=keras.Model(inputs=[inputs,None], outputs=[outputs,outputs2,outputs3,outputs4,outputs5,outputs6], name='Prices_Prediction')

keras.utils.plot_model(model, "my_first_model_with_shape_info.png", show_shapes=True)

In [ ]:
#------------------- Loss and optimizer ----------------------------------------
#got to ensure MeanAbsoluteError it's the good one for our data
loss1 = keras.losses.MeanSquaredError(reduction="auto", name="mean_squared_error")
loss2 = keras.losses.MeanSquaredError(reduction="auto", name="mean_squared_error")
loss3 = keras.losses.MeanSquaredError(reduction="auto", name="mean_squared_error")
loss4 = keras.losses.MeanSquaredError(reduction="auto", name="mean_squared_error")
loss5 = keras.losses.MeanSquaredError(reduction="auto", name="mean_squared_error")
optim=keras.optimizers.Adam(1e-3)
metrics=["accuracy"]

losses={
    "dense": loss1,
    "dense_1": loss2,
    "dense_2": loss3,
    "dense_3": loss4,
    "dense_4": loss5,
}

model.compile(loss=losses, optimizer=optim, metrics=metrics)

print(model.summary())

keras.utils.plot_model(model, "my_first_model_with_shape_info.png", show_shapes=True)

## Model Training ##

In [ ]:
#--------------------------- Assing Y data to losses dictionary -----
y_data={ 
  "dense": trainY_Open,
  "dense_1": trainY_High,
  "dense_2": trainY_Low,
  "dense_3": trainY_Close,
  "dense_4": trainY_Volume,
}

In [ ]:
#------------------------- Training model --------------------------------
history = model.fit(trainX,y=y_data, epochs=200, batch_size=15)

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
losses.plot();

## Model evaluation ##

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,explained_variance_score
import math

In [ ]:


Batch_to_predict=testingX[testingX.shape[0]-2:testingX.shape[0]-1]

print(Batch_to_predict.shape)

print(testingX.shape)

### Evaluation funtion ###

mean square error

Mean square error of testing data

DataSet unseen before

**Nota: Se obtuvo un error del 15% aprox. para los 5 features
para el dataset no visto y ya visto**

In [ ]:
y_data_testing={ 
  "dense": testingY_Open,
  "dense_1": testingY_High,
  "dense_2": testingY_Low,
  "dense_3": testingY_Close,
  "dense_4": testingY_Volume,
}

In [ ]:
mean_square_error_testin_DS=model.evaluate(testingX,y_data_testing,verbose=0)

In [ ]:
mean_square_error_testing_DS_nparray=np.array(mean_square_error_testin_DS)
mean_square_error_testing_DS_nparray=mean_square_error_testing_DS_nparray[0:5]
mean_square_error_testing_DS_nparray_reshaped=np.reshape(mean_square_error_testing_DS_nparray,(1,5))
print(mean_square_error_testing_DS_nparray_reshaped.shape)

In [ ]:
#mean_square_error_training_DS_reshaped=np.array(mean_square_error_training_DS)
testing_DS_scaled_Back = scaler.inverse_transform(mean_square_error_testing_DS_nparray_reshaped)

for i in testing_DS_scaled_Back[0]:
  print(math.sqrt(i))

Mean square error of testing data

DataSet already saw while training

In [ ]:
y_data_taining_eval={ 
  "dense": trainY_Open,
  "dense_1": trainY_High,
  "dense_2": trainY_Low,
  "dense_3": trainY_Close,
  "dense_4": trainY_Volume,
}

In [ ]:
mean_square_error_training_DS=model.evaluate(trainX,y_data_taining_eval,verbose=0)

In [ ]:
mean_square_error_training_DS_nparray=np.array(mean_square_error_training_DS)
mean_square_error_training_DS_nparray=mean_square_error_training_DS_nparray[0:5]
mean_square_error_training_DS_nparray_reshaped=np.reshape(mean_square_error_training_DS_nparray,(1,5))
print(mean_square_error_training_DS_nparray_reshaped.shape)

In [ ]:
#mean_square_error_training_DS_reshaped=np.array(mean_square_error_training_DS)
training_DS_scaled_Back = scaler.inverse_transform(mean_square_error_training_DS_nparray_reshaped)

for i in training_DS_scaled_Back[0]:
  print(math.sqrt(i))

### Evaluation Using the model ###

In [ ]:
N_Days_to_predict=30
Prediction_Saved=[]
#testingX=np.array(testingX)

for i in range(N_Days_to_predict):
  prediction = model.predict(Batch_to_predict) #the input is a 30 units of time batch
  prediction_Reshaped=np.reshape(prediction,(1,1,5))
  Batch_to_predict=np.append(Batch_to_predict,prediction_Reshaped, axis=1)
  Batch_to_predict=np.delete(Batch_to_predict,0,1)
  print(Batch_to_predict.shape)
  Prediction_Saved.append(prediction_Reshaped[0])
  #Perform inverse transformation to rescale back to original range
  #Since we used 5 variables for transform, the inverse expects same dimensions
  #Therefore, let us copy our values 5 times and discard them after inverse transform
#print(Batch_to_predict)
#print(Batch_to_predict)
#print(len(Prediction_Saved))#<----------- 20 days predicted
#print(Prediction_Saved)
#Prediction_Saved=np.array(Prediction_Saved)

#print(Prediction_Saved.shape)


In [ ]:
print(np.array(Prediction_Saved).shape)

In [ ]:
predict_Open=[]
predict_High=[]
predict_Low=[]
predict_Close=[]
predict_Volume=[]

#Splitting data with scaling back
for i in range(N_Days_to_predict):
  y_pred_future = Prediction_Saved[i]
  predict_Open.append(y_pred_future[0][0])
  predict_High.append(y_pred_future[0][1])
  predict_Low.append(y_pred_future[0][2])
  predict_Close.append(y_pred_future[0][3])
  predict_Volume.append(y_pred_future[0][4])



In [ ]:
"""#Spliting and Scaling back data
for i in range(N_Days_to_predict):
  y_pred_future = scaler.inverse_transform(Prediction_Saved[i])
  predict_Open.append(y_pred_future[0][0])
  predict_High.append(y_pred_future[0][1])
  predict_Low.append(y_pred_future[0][2])
  predict_Close.append(y_pred_future[0][3])
  predict_Volume.append(y_pred_future[0][4])"""

## Evaluating Open ##

In [ ]:
#####    Predictiony_open ######
predictY_Open_reshaped = np.reshape(predict_Open,(N_Days_to_predict,1))

In [ ]:
##########   testingY_Open  ############

testingY_Open_reshaped=np.reshape(testingY_Open,(testingY_Open.shape[0],1))
#print(testingY_Open_reshaped.shape)
testingY_Open_reshaped=testingY_Open_reshaped[len(testingY_Open_reshaped)-30:(len(testingY_Open_reshaped)-30)+N_Days_to_predict]
#print(testingY_Open_reshaped)
#testingY_Open_last_part=testingY_Open_reshaped[len(testingY_Open_reshaped)-30:N_Days_to_predict]



In [ ]:
#Open price mean absolute error
mean_absolute_error(testingY_Open_reshaped,predictY_Open_reshaped)

In [ ]:
np.sqrt(mean_squared_error(testingY_Open_reshaped,predictY_Open_reshaped))

In [ ]:
explained_variance_score(testingY_Open_reshaped,predictY_Open_reshaped)

In [ ]:
# Nos prédictions
plt.scatter(testingY_Open_reshaped,predictY_Open_reshaped)

# Prédictions parfaites
plt.plot(testingY_Open_reshaped,predictY_Open_reshaped,'r');

In [ ]:
errors = testingY_Open_reshaped - predictY_Open_reshaped

In [ ]:
sns.distplot(errors);

## Evaluating High ##

In [ ]:
#####  Predictiony_High  ######
predictY_High_reshaped = np.reshape(predict_High,(N_Days_to_predict,1))

In [ ]:
##########   testingY_high  ############
testingY_High_reshaped=np.reshape(testingY_High,(testingY_High.shape[0],1))
#print(testingY_Open_reshaped.shape)
testingY_High_reshaped=testingY_High_reshaped[len(testingY_High_reshaped)-30:(len(testingY_High_reshaped)-30)+N_Days_to_predict]

In [ ]:
#Open price mean absolute error
mean_absolute_error(testingY_High_reshaped,predictY_High_reshaped)

In [ ]:
np.sqrt(mean_squared_error(testingY_High_reshaped,predictY_High_reshaped))

In [ ]:
explained_variance_score(testingY_High_reshaped,predictY_High_reshaped)

In [ ]:
# Nos prédictions
plt.scatter(testingY_High_reshaped,predictY_High_reshaped)

# Prédictions parfaites
plt.plot(testingY_High_reshaped,predictY_High_reshaped,'r');

In [ ]:
errors = testingY_High_reshaped - predictY_High_reshaped

In [ ]:
sns.distplot(errors);

## Evaluating Low ##

In [ ]:
#####  Predictiony_Low  ######
predictY_Low_reshaped = np.reshape(predict_Low,(N_Days_to_predict,1))

In [ ]:
##########   testingY_high  ############
testingY_Low_reshaped=np.reshape(testingY_Low,(testingY_Low.shape[0],1))
#print(testingY_Open_reshaped.shape)
testingY_Low_reshaped=testingY_Low_reshaped[len(testingY_Low_reshaped)-30:(len(testingY_Low_reshaped)-30)+N_Days_to_predict]

In [ ]:
#Open price mean absolute error
mean_absolute_error(testingY_Low_reshaped,predictY_Low_reshaped)

In [ ]:
np.sqrt(mean_squared_error(testingY_Low_reshaped,predictY_Low_reshaped))

In [ ]:
explained_variance_score(testingY_Low_reshaped,predictY_Low_reshaped)

In [ ]:
# Nos prédictions
plt.scatter(testingY_Low_reshaped,predictY_Low_reshaped)

# Prédictions parfaites
plt.plot(testingY_Low_reshaped,predictY_Low_reshaped,'r');

In [ ]:
errors = testingY_Low_reshaped - predictY_Low_reshaped

In [ ]:
sns.distplot(errors);

## Evaluating Close ##

In [ ]:
#####  Predictiony_Close  ######
predictY_Close_reshaped = np.reshape(predict_Close,(N_Days_to_predict,1))

In [ ]:
##########   testingY_high  ############
testingY_Close_reshaped=np.reshape(testingY_Close,(testingY_Close.shape[0],1))
#print(testingY_Open_reshaped.shape)
testingY_Close_reshaped=testingY_Close_reshaped[len(testingY_Close_reshaped)-30:(len(testingY_Close_reshaped)-30)+N_Days_to_predict]

In [ ]:
#Open price mean absolute error
mean_absolute_error(testingY_Close_reshaped,predictY_Close_reshaped)

In [ ]:
np.sqrt(mean_squared_error(testingY_Close_reshaped,predictY_Close_reshaped))

In [ ]:
explained_variance_score(testingY_Close_reshaped,predictY_Close_reshaped)

In [ ]:
# Nos prédictions
plt.scatter(testingY_Close_reshaped,predictY_Close_reshaped)

# Prédictions parfaites
plt.plot(testingY_Close_reshaped,predictY_Close_reshaped,'r');

In [ ]:
errors = testingY_Close_reshaped - predictY_Close_reshaped

In [ ]:
sns.distplot(errors);

## Evaluating Volume ##

In [ ]:
#####  Predictiony_Close  ######
predictY_Volume_reshaped = np.reshape(predict_Volume,(N_Days_to_predict,1))

In [ ]:
##########   testingY_high  ############
testingY_Volume_reshaped=np.reshape(testingY_Volume,(testingY_Volume.shape[0],1))
#print(testingY_Open_reshaped.shape)
testingY_Volume_reshaped=testingY_Volume_reshaped[len(testingY_Volume_reshaped)-30:(len(testingY_Volume_reshaped)-30)+N_Days_to_predict]

In [ ]:
#Open price mean absolute error
mean_absolute_error(testingY_Volume_reshaped,predictY_Volume_reshaped)

In [ ]:
np.sqrt(mean_squared_error(testingY_Volume_reshaped,predictY_Volume_reshaped))

In [ ]:
explained_variance_score(testingY_Volume_reshaped,predictY_Volume_reshaped)

In [ ]:
# Nos prédictions
plt.scatter(testingY_Volume_reshaped,predictY_Volume_reshaped)

# Prédictions parfaites
plt.plot(testingY_Volume_reshaped,predictY_Volume_reshaped,'r');

In [ ]:
errors = testingY_Volume_reshaped - predictY_Volume_reshaped

In [ ]:
sns.distplot(errors);

## Getting the candle chart ##

In [ ]:
BatchUsed_Dates=DS_dates_finished_X[len(DS_dates_finished_X)-2:len(DS_dates_finished_X)-1]

lastTimedate=BatchUsed_Dates[0][BatchUsed_Dates.shape[1]-1:]
lastTimedate=lastTimedate[0]

print((lastTimedate))



In [ ]:

import pandas as pd
import numpy as np

BatchForcasted_Dates=[]
for i in range(0,N_Days_to_predict):
  BatchForcasted_Dates.append(pd.Timestamp(np.datetime64(lastTimedate)))
  lastTimedate=np.datetime64(lastTimedate) + np.timedelta64(1, 'm')
print(BatchForcasted_Dates)




In [ ]:
AllPrediction_DS_scaled_Back=[]
for i in Prediction_Saved:
  AllPrediction_DS_scaled_Back.append(scaler.inverse_transform(i))

In [ ]:
print(np.array(AllPrediction_DS_scaled_Back).shape)
predict_Open=[]
predict_High=[]
predict_Low=[]
predict_Close=[]
predict_Volume=[]

#Splitting data with scaling back
for i in range(N_Days_to_predict):
  y_pred_future = AllPrediction_DS_scaled_Back[i]
  predict_Open.append(y_pred_future[0][0])
  predict_High.append(y_pred_future[0][1])
  predict_Low.append(y_pred_future[0][2])
  predict_Close.append(y_pred_future[0][3])
  predict_Volume.append(y_pred_future[0][4])

In [ ]:
#--------  data shape it's (x days, 6 columns)
# Convert timestamp to date
df_forecast = pd.DataFrame({'Open':predict_Open,'High':predict_High, 'Low':predict_Low,'Close':predict_Close,'Volume':predict_Volume}, index=BatchForcasted_Dates)

df_forecast.index.name="Date"



In [ ]:
!pip install mplfinance

In [ ]:
import mplfinance as mpf
title_chart="Twtr"
mpf.plot(df_forecast, type='candle',title=title_chart, style='charles')